<div style="background:#F5F7FA; height:100px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Want to do more?</span><span style="border: 1px solid #3d70b2;padding: 15px;float:right;margin-right:40px; color:#3d70b2; "><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
<span style="color:#5A6872;"> Try out this notebook with your free trial of IBM Watson Studio.</span>
</div>

<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>From Local Spark MLlib Model to Cloud with Watson Machine Learning Client</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://www-03.ibm.com/services/learning/images/journey_watson_wat_machine_learning_600x280.jpg" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to work with **``watson-machine-learning-client``** library. This notebook introduces commands for getting data, model persistance to Watson Machine Learning repository, model deployment and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.5 and Apache Spark 2.1.

You will use publicly available data "GoSales Transactions for Naive Bayes Model" about anonymous outdoor equipment purchase to predict clients' interests in terms of product line like: golf accessories, camping equipement and others.


## Learning goals

The learning goals of this notebook are:
- Load and explore data
- Prepare data for training and evaluation
- Create spark ml pipeline
- Train and evaluate model
- Persist local model in Watson Machine Learning repository using client library
- Deploy model for online scoring using client library
- Score sample records using client library


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create an Apache Spark machine learning model](#model)
4.	[Persist created model using api client](#persistance)
5.	[Deploy and score in a Cloud using api client](#scoring)
6.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/machine-learning/" target="_blank" rel="noopener noreferrer">Watson Machine Learning Service</a> instance (a free plan is offered). 
-  Make sure that you are using a Spark 2.1 kernel.

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration.

Load the data to the Spark DataFrame by using `wget` to upload the data to gpfs and then `read` method. 

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.

In [ ]:
#!pip install wget --upgrade

In [1]:
import wget
import json
import os

In [2]:
filename = 'GoSales_Tx_NaiveBayes.csv'

if not os.path.isfile(filename):
    link_to_data = 'https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600'
    filename = wget.download(link_to_data)

print(filename)

GoSales_Tx_NaiveBayes.csv


The csv file `GoSales_Tx_NaiveBayes.csv` is availble on gpfs now. Load the file to Apache® Spark DataFrame using code below.

In [3]:
#spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(filename)

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  print top ten records
-  count all records

In [4]:
df_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



As you can see, the data contains five fields. PRODUCT_LINE field is the one we would like to predict (label).

In [5]:
df_data.show()

+--------------------+------+---+--------------+------------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|
+--------------------+------+---+--------------+------------+
|Personal Accessories|     M| 27|        Single|Professional|
|Personal Accessories|     F| 39|       Married|       Other|
|Mountaineering Eq...|     F| 39|       Married|       Other|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|
|      Golf Equipment|     M| 45|       Married|     Retired|
|      Golf Equipment|     M| 45|       Married|     Retired|
|   Camping Equipment|     F| 39|       Married|       Other|
|   Camping Equipment|     F| 49|       Married|       Other|
|  Outdoor Protection|     F| 49|       Married|       Other|
|      Golf Equipment|     M| 47|       Married|     Retired|
|      Golf Equipment|     M| 47|       Married|     Retired|
|Mountaineering Eq...|     M| 21|        Single|      Retail|
|Personal Accessories|     F| 66|       Married|       Other|
|   Camp

In [6]:
df_data.count()

60252

As you can see, the data set contains 60252 records.

<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [7]:
splitted_data = df_data.randomSplit([0.8, 0.2], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))

Number of training records: 48176
Number of testing records : 12076


As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.

### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [8]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the string fields to numeric ones by using the StringIndexer transformer.

In [9]:
stringIndexer_label = StringIndexer(inputCol="PRODUCT_LINE", outputCol="label").fit(df_data)
stringIndexer_prof = StringIndexer(inputCol="PROFESSION", outputCol="PROFESSION_IX")
stringIndexer_gend = StringIndexer(inputCol="GENDER", outputCol="GENDER_IX")
stringIndexer_mar = StringIndexer(inputCol="MARITAL_STATUS", outputCol="MARITAL_STATUS_IX")

In the following step, create a feature vector by combining all features together.

In [10]:
vectorAssembler_features = VectorAssembler(inputCols=["GENDER_IX", "AGE", "MARITAL_STATUS_IX", "PROFESSION_IX"], outputCol="features")

Next, define estimators you want to use for classification. Random Forest is used in the following example.

In [11]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, indexed labels back to original labels.

In [12]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [13]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_prof, stringIndexer_gend, stringIndexer_mar, vectorAssembler_features, rf, labelConverter])

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [14]:
model_rf = pipeline_rf.fit(train_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [15]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)

print("Accuracy = %g" % accuracy)

Accuracy = 0.584796


You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 4. Persist created model with ``watson-machine-learning-client``

In this section you will learn how to store your model in Watson Machine Learning repository by using common python client.

#### First, you must install client libraries from pypi

In [16]:
!pip install watson-machine-learning-client --upgrade

    100% |████████████████████████████████| 450kB 1.3MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 2.6MB/s eta 0:00:01
Requirement already up-to-date: tabulate in /usr/local/src/conda3_runtime.v28/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: urllib3 in /usr/local/src/conda3_runtime.v28/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: certifi in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s778-bfb6f75aebc10f-9bb95b1f072f/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
    100% |████████████████████████████████| 25.7MB 46kB/s  eta 0:00:01
Requirement already up-to-date: requests in /usr/local/src/conda3_runtime.v28/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: python-dateutil>=2 in /usr/local/src/conda3_r

Import installed client by running below code.

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**You can check sphinx documentation for watson-machine-learning-client by visiting the <a href="https://wml-api-pyclient.mybluemix.net" target="_blank" rel="noopener noreferrer">documentation webpage</a>.**

### 4.1: Work with your instance

Authenticate to Watson Machine Learning service on IBM Cloud.

**Action**: Put authentication information from your instance of Watson Machine Learning service here.</div>

In [18]:
wml_credentials={
  "url": "***",
  "access_key": "***",
  "username": "***",
  "password": "***",
  "instance_id": "***"
}

**Tip**: Credentials can be found on **Service Credentials** tab of service instance created in IBM Cloud. If you cannot see **instance_id** field in **Serice Credentials** generate new credentials by pressing **New credential (+)** button. 

#### Create API client by running below code.

In [19]:
client = WatsonMachineLearningAPIClient(wml_credentials)

#### Get instance details.

In [20]:
instance_details = client.service_instance.get_details()

print(json.dumps(instance_details, indent=2))

{
  "metadata": {
    "created_at": "2018-02-19T14:57:53.275Z",
    "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8f11cfac-57ca-4a0c-9815-7abbd7a72491",
    "modified_at": "2018-02-19T14:57:53.275Z",
    "guid": "8f11cfac-57ca-4a0c-9815-7abbd7a72491"
  },
  "entity": {
    "region": "us-south",
    "account": {
      "id": "e2cb778ce7da6d8945e99e5f02b48a43",
      "type": "TRIAL",
      "name": "Melanie Manley's Account"
    },
    "deployments": {
      "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8f11cfac-57ca-4a0c-9815-7abbd7a72491/deployments"
    },
    "source": "Bluemix",
    "status": "Active",
    "plan_id": "3f6acf43-ede8-413a-ac69-f8af3bb0cbfe",
    "published_models": {
      "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8f11cfac-57ca-4a0c-9815-7abbd7a72491/published_models"
    },
    "organization_guid": "411fda6f-bb89-452d-98f2-fb852198f154",
    "plan": "lite",
    "space_guid": "f6ef3ba5-8f4e-4614-a10f-9bb95b1f072f",
    "u

### 4.2: Publish model

#### Publish model in Watson Machine Learning repository on Cloud.

In [21]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "ibm@ibm.com", 
               client.repository.ModelMetaNames.NAME: "LOCALLY created Product Line Prediction model"}
published_model = client.repository.store_model(model=model_rf, pipeline=pipeline_rf, meta_props=model_props, training_data=train_data)

### 4.3: Get model details

In [22]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "metadata": {
    "created_at": "2018-03-06T09:24:08.939Z",
    "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8f11cfac-57ca-4a0c-9815-7abbd7a72491/published_models/d82bced1-f287-4045-9e30-551d87e2ed32",
    "modified_at": "2018-03-06T09:24:09.009Z",
    "guid": "d82bced1-f287-4045-9e30-551d87e2ed32"
  },
  "entity": {
    "latest_version": {
      "created_at": "2018-03-06T09:24:09.009Z",
      "url": "https://ibm-watson-ml.mybluemix.net/v3/ml_assets/models/d82bced1-f287-4045-9e30-551d87e2ed32/versions/a41c29db-de4e-4dd7-b96d-9d8cc62cd2fd",
      "guid": "a41c29db-de4e-4dd7-b96d-9d8cc62cd2fd"
    },
    "runtime_environment": "spark-2.1",
    "name": "LOCALLY created Product Line Prediction model",
    "learning_configuration_url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8f11cfac-57ca-4a0c-9815-7abbd7a72491/published_models/d82bced1-f287-4045-9e30-551d87e2ed32/learning_configuration",
    "model_type": "mllib-2.1",
    "input_data_schema": {
      "type

#### Get all models

In [23]:
client.repository.list_models()

------------------------------------  ---------------------------------------------  ------------------------  ---------  -----
GUID                                  NAME                                           CREATED                   FRAMEWORK  TYPE
c19ea489-1674-4883-bcbd-68f0089ef7dc  Customer Churn Model                           2018-02-19T15:01:13.485Z  mllib-2.0  model
d9083d24-67f6-418e-869d-4fe344f7d9e5  Customer Churn Model                           2018-02-20T07:20:03.989Z  mllib-2.1  model
7bce0af2-3d3a-4163-9bce-f5303c729d07  Customer Churn Model                           2018-02-20T09:13:27.542Z  mllib-2.1  model
1698755c-ae01-4ab9-80b3-4f5d4d92bcc3  VIOLATIONS_SCALA211_SPARK20                    2018-02-20T13:25:01.431Z  mllib-2.1  model
d82bced1-f287-4045-9e30-551d87e2ed32  LOCALLY created Product Line Prediction model  2018-03-06T09:24:08.939Z  mllib-2.1  model
------------------------------------  ---------------------------------------------  --------------------

### 4.3: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [24]:
loaded_model = client.repository.load(published_model_uid)

You can pass test_data to loaded model ``transform()`` method to make sure that model has been loaded correctly.

In [25]:
test_predictions = loaded_model.transform(test_data)

In [26]:
test_predictions.select('probability', 'predictedLabel').show(n=3, truncate=False)

+------------------------------------------------------------------------------------------------------+--------------------+
|probability                                                                                           |predictedLabel      |
+------------------------------------------------------------------------------------------------------+--------------------+
|[0.27034892863178805,0.5134958093759995,0.1838796533599044,0.010175481590873187,0.022100127041434846] |Personal Accessories|
|[0.15058550023691625,0.6928114355885882,0.12745076465565847,0.009714176086733068,0.019438123432104045]|Personal Accessories|
|[0.5153723402231977,0.3652003462475749,0.08273067150628524,0.017825283072460344,0.01887135895048206]  |Camping Equipment   |
+------------------------------------------------------------------------------------------------------+--------------------+
only showing top 3 rows



As you can see the loaded model works. You have already learned how save and load the model from Watson Machine Learning repository.

### 4.4: Delete model

You can delete published model from Watson Machine Learning repository using below code. The code is commented out at this stage since the model will be needed later for deployment.

<a id="scoring"></a>
## 5. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record by using the Watson Machine Learning Client. 

### 5.1: Create online deployment

In [27]:
created_deployment = client.deployments.create(published_model_uid, name="Product line prediction")

Now you can print an online scoring endpoint. 

In [28]:
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)

print(scoring_endpoint)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8f11cfac-57ca-4a0c-9815-7abbd7a72491/published_models/d82bced1-f287-4045-9e30-551d87e2ed32/deployments/4e8c133a-f17c-4442-b61c-c2fb902db14d/online


### 5.2: Get deployments

In [29]:
client.deployments.list()

------------------------------------  -------------------------  ------  ------------------------  ---------
GUID                                  NAME                       TYPE    CREATED                   FRAMEWORK
4e8c133a-f17c-4442-b61c-c2fb902db14d  Product line prediction    online  2018-03-06T09:24:20.536Z  mllib-2.1
bb3cb635-7d08-4115-9592-113ff94aabde  Customer Churn Prediction  batch   2018-02-19T15:11:40.938Z  mllib-2.0
------------------------------------  -------------------------  ------  ------------------------  ---------


You can see that your model is deployed, so you can score new data.

### 5.3: Score

You can use below method to do test scoring request against deployed model.

**Action**: Prepare scoring payload with records to score.

In [30]:
scoring_payload = {"fields": ["GENDER","AGE","MARITAL_STATUS","PROFESSION"],"values": [["M",23,"Single","Student"],["M",55,"Single","Executive"]]}

Use ``client.deployments.score()`` method to run scoring.

In [31]:
predictions = client.deployments.score(scoring_endpoint, scoring_payload)

In [32]:
print(json.dumps(predictions, indent=2))

{
  "fields": [
    "GENDER",
    "AGE",
    "MARITAL_STATUS",
    "PROFESSION",
    "PRODUCT_LINE",
    "label",
    "PROFESSION_IX",
    "GENDER_IX",
    "MARITAL_STATUS_IX",
    "features",
    "rawPrediction",
    "probability",
    "prediction",
    "predictedLabel"
  ],
  "values": [
    [
      "M",
      23,
      "Single",
      "Student",
      "Camping Equipment",
      0.0,
      6.0,
      0.0,
      1.0,
      [
        0.0,
        23.0,
        1.0,
        6.0
      ],
      [
        5.258712981131878,
        6.723421733525897,
        6.0132018750333724,
        0.218060296213069,
        1.7866031140957863
      ],
      [
        0.26293564905659383,
        0.3361710866762948,
        0.3006600937516686,
        0.010903014810653448,
        0.0893301557047893
      ],
      1.0,
      "Personal Accessories"
    ],
    [
      "M",
      55,
      "Single",
      "Executive",
      "Camping Equipment",
      0.0,
      3.0,
      0.0,
      1.0,
      [
        0

### 5.4: Delete deployment

Use the following method to delete deployment.

In [33]:
client.deployments.delete(client.deployments.get_uid(created_deployment))

You can check number of depoyments by running ``client.deployment.get()`` method.

In [34]:
client.deployments.list()

------------------------------------  -------------------------  -----  ------------------------  ---------
GUID                                  NAME                       TYPE   CREATED                   FRAMEWORK
bb3cb635-7d08-4115-9592-113ff94aabde  Customer Churn Prediction  batch  2018-02-19T15:11:40.938Z  mllib-2.0
------------------------------------  -------------------------  -----  ------------------------  ---------


### 5.5: Delete model

In [35]:
client.repository.delete(published_model_uid)

You can check number of your models by running below ``client.repository.get_details()`` method.

In [36]:
client.repository.list_models()

------------------------------------  ---------------------------  ------------------------  ---------  -----
GUID                                  NAME                         CREATED                   FRAMEWORK  TYPE
c19ea489-1674-4883-bcbd-68f0089ef7dc  Customer Churn Model         2018-02-19T15:01:13.485Z  mllib-2.0  model
d9083d24-67f6-418e-869d-4fe344f7d9e5  Customer Churn Model         2018-02-20T07:20:03.989Z  mllib-2.1  model
7bce0af2-3d3a-4163-9bce-f5303c729d07  Customer Churn Model         2018-02-20T09:13:27.542Z  mllib-2.1  model
1698755c-ae01-4ab9-80b3-4f5d4d92bcc3  VIOLATIONS_SCALA211_SPARK20  2018-02-20T13:25:01.431Z  mllib-2.1  model
------------------------------------  ---------------------------  ------------------------  ---------  -----


<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our <a href="https://console.bluemix.net/docs/services/PredictiveModeling/index.html#WMLgettingstarted" target="_blank" rel="noopener noreferrer">Online Documentation</a> for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is a Automation Architect and Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017, 2018 IBM. This notebook and its source code are released under the terms of the MIT License.